**Problem set 1**
**Subject: IO**
**Start Date 01/12/2025**

**Problem 1**

In [1]:
using CSV, DataFrames, RegressionTables

#Input path
data_path = "/Users/jorditorresvallverdu/Library/Mobile Documents/com~apple~CloudDocs/tse/year2/term1/io/part2/data_ps1.csv" ##-->CHANGE JUST THIS PART MANUALLY!

#Output path
export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/" ##Change this manually too

main = CSV.read(data_path, DataFrame)

Row,yr,prefid,firmid,m,k,age,acquired,cost_m,q,rev,l,cost_l,cost_k
,Int64,Int64,Int64,Int64?,Int64?,Int64,Int64,Float64?,Float64,Float64,Int64?,Float64?,Float64?
1,1917,1,1,174866,120960,0,0,4.81021e5,1.55838e5,385355.0,1045968,49963.2,missing
2,1918,1,1,225000,120600,1,0,9.72158e5,412452.0,2.01e6,1359765,84420.0,missing
3,1919,1,1,225266,199248,2,0,1.16798e6,8.82063e5,5.14022e6,4956336,4.60354e5,missing
4,1920,1,1,383289,306440,3,0,2.14914e6,1.11663e6,5.15899e6,5278592,215085.0,missing
5,1896,1,2,33580,5772,18,0,38996.5,24626.3,55610.1,522144,2925.96,missing
6,1896,4,4,502159,112346,4,0,5.83157e5,3.97768e5,789222.0,8201556,53398.6,16698.8
7,1897,4,4,532756,144058,5,0,6.45381e5,461018.0,8.54804e5,8520094,78892.8,43161.2
8,1898,4,4,190027,49268,6,0,2.19785e5,150335.0,0.0,2707342,23877.5,30933.8
9,1896,16,5,843772,243090,7,0,9.79872e5,1.00331e6,1.35233e6,13692294,1.02388e5,39081.8


In [2]:
#1.a
using Statistics

#define variables

l= main.l
w= main.cost_l
rev= main.rev
k= main.k
r= main.cost_k
y= main.q

beta_l= ifelse.(rev.>0, (w) ./ (rev), missing) 

beta_l_total = mean(skipmissing(beta_l))
beta_l_std= std(skipmissing(beta_l))
beta_l_median= median(skipmissing(beta_l))

beta_1_res = vcat(beta_l_total, beta_l_std, beta_l_median)


3-element Vector{Float64}:
 0.0689133994639132
 0.28532078968029445
 0.050841608736857224

In [3]:
#1.b 
using Statistics

total_cost = w .+  r

idx = findall(x -> !ismissing(x) && x > 0, total_cost)

beta_l_2 = (w[idx]) ./ total_cost[idx]


beta_l_total_2 = mean(beta_l_2)
beta_l_std_2= std(beta_l_2)
beta_l_median_2= median(beta_l_2)



beta_2_res = vcat(beta_l_total_2, beta_l_std_2, beta_l_median_2)




3-element Vector{Float64}:
 0.5404447821075812
 0.17975438640775313
 0.5693124616887979

In [4]:
#1.c 

markup= rev[idx] ./ total_cost[idx]

markup_mean = mean(markup)
markup_std= std(markup)
markup_median= median(markup)

markup_mean


markup_res= vcat(markup_mean,markup_std, markup_median )

3-element Vector{Float64}:
 16.184801788871056
 16.404163279492682
 12.086569058219947

In [5]:
using DataFrames

results_1 = DataFrame(
    Statistic = ["Mean", "Std. Dev.", "Median"],
    β_l_rev   = beta_1_res,
    β_l_cost  = beta_2_res,
    Markup    = markup_res
)

show(stdout, MIME("text/latex"), results_1)


\begin{tabular}{r|cccc}
	& Statistic & β\_l\_rev & β\_l\_cost & Markup\\
	\hline
	& String & Float64 & Float64 & Float64\\
	\hline
	1 & Mean & 0.0689134 & 0.540445 & 16.1848 \\
	2 & Std. Dev. & 0.285321 & 0.179754 & 16.4042 \\
	3 & Median & 0.0508416 & 0.569312 & 12.0866 \\
\end{tabular}


14

**Exercise 2**


## Two-step GMM estimation of a production function with dynamic productivity and acquisitions

We estimate a Cobb–Douglas production function in logs:
$$
y_{ft} = \beta_0 + \beta_\ell \ell_{ft} + \beta_k k_{ft} + \omega_{ft},
$$
where $y_{ft}=\log Q_{ft}$, $\ell_{ft}=\log L_{ft}$, $k_{ft}=\log K_{ft}$, and $\omega_{ft}$ is firm-level productivity observed by the firm but not by the econometrician.

---

### Productivity process with acquisitions

Productivity follows an AR(1) process augmented with acquisitions:
$$
\omega_{ft} = \rho\,\omega_{f,t-1} + \beta_a A_{f,t-1} + \nu_{ft},
$$
where $A_{f,t-1}$ is an indicator for being acquired in the previous period, and $\nu_{ft}$ is an innovation uncorrelated with information dated $t-1$.

To enforce stationarity during estimation, we reparametrize
$$
\rho = \tanh(\psi), \qquad \rho \in (-1,1),
$$
and estimate $\psi$ instead of $\rho$.

---

### Constructed productivity and innovation

Given parameters $\theta=(\beta_0,\beta_\ell,\beta_k,\psi,\beta_a)$, define
$$
\widehat{\omega}_{ft}(\theta)
= y_{ft} - \beta_0 - \beta_\ell \ell_{ft} - \beta_k k_{ft},
$$
$$
\widehat{\omega}_{f,t-1}(\theta)
= y_{f,t-1} - \beta_0 - \beta_\ell \ell_{f,t-1} - \beta_k k_{f,t-1}.
$$

The implied innovation is
$$
\widehat{\nu}_{ft}(\theta)
= \widehat{\omega}_{ft}(\theta)
- \rho\,\widehat{\omega}_{f,t-1}(\theta)
- \beta_a A_{f,t-1}.
$$

---

### Moment conditions

Let $Z_{ft}\in\mathbb{R}^q$ be a vector of instruments dated $t-1$ or earlier.  
The identifying assumption is
$$
\mathbb{E}\!\left[ Z_{ft}\,\nu_{ft} \right] = 0.
$$

The sample moment vector is
$$
g_n(\theta)
= \frac{1}{n} \sum_{i=1}^n Z_i \widehat{\nu}_i(\theta)
= \frac{1}{n} Z^\top \widehat{\nu}(\theta)
\in \mathbb{R}^q.
$$

---

### GMM objective function

For a positive semi-definite weighting matrix $W$,
$$
Q_n(\theta;W)
= n \, g_n(\theta)^\top W \, g_n(\theta).
$$

---

### Two-step GMM procedure

**Step 1 (initial estimation).**  
Use $W_1 = I_q$ and compute
$$
\hat{\theta}_1
= \arg\min_{\theta} Q_n(\theta;W_1).
$$

**Robust moment covariance.**  
Define the moment contributions
$$
m_i(\theta) = Z_i \widehat{\nu}_i(\theta).
$$
A heteroskedasticity-robust estimator of the covariance matrix is
$$
\hat{S}_1
= \frac{1}{n} \sum_{i=1}^n m_i(\hat{\theta}_1)m_i(\hat{\theta}_1)^\top.
$$

**Step 2 (efficient estimation).**  
Use the optimal weighting matrix
$$
W_2 = \hat{S}_1^{\dagger},
$$
where $(\cdot)^\dagger$ denotes the Moore–Penrose pseudoinverse, and compute
$$
\hat{\theta}_2
= \arg\min_{\theta} Q_n(\theta;W_2),
$$
starting the optimization from $\hat{\theta}_1$.

---

### Robust (sandwich) standard errors

Let
$$
D(\theta)
= \frac{\partial g_n(\theta)}{\partial \theta^\top}
$$
be the Jacobian of the sample moments. Recompute
$$
\hat{S}_2
= \frac{1}{n} \sum_{i=1}^n m_i(\hat{\theta}_2)m_i(\hat{\theta}_2)^\top.
$$

The asymptotic variance of $\hat{\theta}_2$ is estimated by
$$
\widehat{\mathrm{Var}}(\hat{\theta}_2)
=
\frac{1}{n}
\left(D^\top W_2 D\right)^{-1}
\left(D^\top W_2 \hat{S}_2 W_2 D\right)
\left(D^\top W_2 D\right)^{-1}.
$$

Standard errors are given by the square roots of the diagonal elements.


In [6]:
#dayacleaning
using ShiftedArrays

main_raw = deepcopy(main)

function prepare_data(main_raw)
    df = deepcopy(main_raw)

    sort!(df, [:firmid, :yr])

    df = transform(
        groupby(df, :firmid),
        :l => (x -> ShiftedArrays.lag(x, 1)) => :l_lag1,
        :l => (x -> ShiftedArrays.lag(x, 2)) => :l_lag2,
        :k => (x -> ShiftedArrays.lag(x, 1)) => :k_lag1,
        :acquired => (x -> ShiftedArrays.lag(x, 1)) => :acquired_lag1,
        :acquired => (x -> ShiftedArrays.lag(x, 2)) => :acquired_lag2,
        :q => (x -> ShiftedArrays.lag(x, 1)) => :y_lag,
        :age => (x -> ShiftedArrays.lag(x, 1)) => :age_lag

    )

    needed = [:l, :k, :q, :l_lag1, :l_lag2, :k_lag1, :acquired_lag1, :acquired_lag2, :y_lag, :age_lag]
    df = dropmissing(df, needed)

    df = filter(row ->
        row.l > 0 &&
        row.k > 0 &&
        row.q > 0 &&
        row.y_lag > 0 &&
        row.l_lag1 > 0 &&
        row.l_lag2 > 0 &&
        row.k_lag1 > 0,
        df
    )

    return df
end


#de-mean to eliminate time-FE?? kills variation...no


data = prepare_data(main_raw)


#make sure order is maintained of the vectors after sort
l= data.l
w= data.cost_l
rev= data.rev
k= data.k
r= data.cost_k
y= data.q

age_lag= data.age_lag

acquired_lag1= data.acquired_lag1
acquired_lag2=data.acquired_lag2
y_lag= data.y_lag


# turn variables into log
log_output= log.(y)
log_lagoutput= log.(y_lag)
log_l= log.(l)
log_k= log.(k)

log_l_lag1= log.(data.l_lag1)
log_l_lag2= log.(data.l_lag2)
log_k_lag1= log.(data.k_lag1)


# n

n = nrow(data)

# Matrix of Z, this is assuming At-1 is exogeneous

Z = hcat(
    ones(n),
    log_l_lag1,
    log_k,
    log_k_lag1,
    acquired_lag1
)

n

901

In [7]:
using LinearAlgebra
#outdated

#guess of values goes in theta. 
function gmm_estim(θ::AbstractVector, Z::AbstractMatrix, n)

    β0, βl, βk, ρ, βa= θ

    ω_t= @. log_output - β0 - βl*log_l - βk*log_k #nx1
    ω_t_lag= @. log_lagoutput- β0 - βl* log_l_lag1 -  βk*log_k_lag1 #nx1

    ν_hat= @. ω_t- ρ*ω_t_lag - βa*acquired_lag1 #nx1

    gn= (Z'*ν_hat)/n #kxn nx1-> kx1 moments

    W=I(length(gn))

    J = n * dot(gn, gn) #we minimize this to find θ
    return J

end 

gmm_estim (generic function with 1 method)

In [8]:
#Set reasonable starting values, starting from OLS:

using FixedEffectModels
X = hcat(
    ones(length(log_l)),
    log_l,
    log_k)
    


B_ols= (X'*X)\(X'*log_output)


df = DataFrame(
    log_output = log_output,
    log_l = log_l,
    log_k = log_k)

reg_ols = reg(df, @formula(log_output ~ log_l + log_k ))

coef_ols = coef(reg_ols)
se_ols   = stderror(reg_ols)


3-element Vector{Float64}:
 0.31920198691190654
 0.046003678780534595
 0.05075544190216769

In [9]:
using Optim

ψ0 = atanh(0.7)
θ0 = [B_ols[1], B_ols[2], B_ols[3], ψ0, 0.0]


#θ0 = [B_ols[1], B_ols[2], B_ols[3], 0.8, 0.5]   # example 

obj(θ) = gmm_estim(θ, Z, n)


res1_test = optimize(obj, θ0, NelderMead())
θ1_test = Optim.minimizer(res1_test)


5-element Vector{Float64}:
 -3.2966993118203938
  0.38476640313634863
  0.9073230678761943
 -1.0811221299086948
  0.1079398999065807

In [10]:
#two-step GMM, step 1.

function moments(θ::AbstractVector, Z::AbstractMatrix, n)

    β0, βl, βk, ψ, βa = θ

    ρ = tanh(ψ)


    ω_t = @. log_output - β0 - βl*log_l - βk*log_k
    ω_t_lag = @. log_lagoutput - β0 - βl*log_l_lag1 - βk*log_k_lag1

    ν_hat = @. ω_t - ρ*ω_t_lag - βa*acquired_lag1   # n×1

    g = (Z' * ν_hat) / n   # q×1 vector of moments

    return g, ν_hat
end


function gmm_objective(θ, Z, n, W)

    g, _=moments(θ, Z, n)
    return n * dot(g, W * g) #difference?

end 

function s_robust(θ, Z, n)

    _, ν= moments(θ, Z, n)

    M= Z.*ν

    return (M'*M) /n

end 

using ForwardDiff, LinearAlgebra

function compute_se(Z, n, θ2, W2, S1)
    gfun(θ) = moments(θ, Z, n)[1]

    D= ForwardDiff.jacobian(gfun,θ2)

    A = D' * W2 * D
    B = D' * W2 * S1 * W2 * D

    S1 = 0.5 * (S1 + S1')
    W2 = 0.5 * (W2 + W2')
    A  = 0.5 * (A  + A')


    V = inv(A) * B * inv(A) / n
    se = sqrt.(diag(V))

    return se 


end 


#full fledge gmm 
function gmm(Z, n)

    q = size(Z, 2)
    W1 = Matrix(I, q, q)

    # first step
    obj1(θ) = gmm_objective(θ, Z, n, W1)
    res1 = optimize(obj1, θ0, NelderMead())
    θ1 = Optim.minimizer(res1)

    # optimal weighting matrix (computed at θ1)
    S1 = s_robust(θ1, Z, n)
    W2 = pinv(S1)

    # second step
    obj2(θ) = gmm_objective(θ, Z, n, W2)
    res2 = optimize(obj2, θ1, NelderMead())   # start from θ1
    θ2 = Optim.minimizer(res2)

    # recompute meat at θ2 for SEs
    S2 = s_robust(θ2, Z, n)

    # standard errors
    se = compute_se(Z, n, θ2, W2, S2)

    #return

    param = hcat(
    θ2,
    se)

    return param


end 



param = gmm(Z, n)



5×2 Matrix{Float64}:
  -3.30106   0.282123
   0.385612  0.0533172
   0.906453  0.060775
 -17.2182    1.0864e14
   2.20214   0.951601

In [11]:
#compare GMM with OLS

table_gmm_ols = DataFrame(
    Parameter = ["β₀", "β_l", "β_k", "ψ", "β_a"],

    OLS_Estimate = [
        coef_ols[1],
        coef_ols[2],
        coef_ols[3],
        missing,
        missing
    ],

    OLS_SE = [
        se_ols[1],
        se_ols[2],
        se_ols[3],
        missing,
        missing
    ],

    GMM_Estimate = param[:, 1],
    GMM_SE       = param[:, 2]
)
show(stdout, MIME("text/latex"), table_gmm_ols)



\begin{tabular}{r|ccccc}
	& Parameter & OLS\_Estimate & OLS\_SE & GMM\_Estimate & GMM\_SE\\
	\hline
	& String & Float64? & Float64? & Float64 & Float64\\
	\hline
	1 & β₀ & -2.54257 & 0.319202 & -3.30106 & 0.282123 \\
	2 & β\_l & 0.401443 & 0.0460037 & 0.385612 & 0.0533172 \\
	3 & β\_k & 0.822638 & 0.0507554 & 0.906453 & 0.060775 \\
	4 & ψ & \emph{missing} & \emph{missing} & -17.2182 & 1.0864e14 \\
	5 & β\_a & \emph{missing} & \emph{missing} & 2.20214 & 0.951601 \\
\end{tabular}


14

here $ψ$ is $tanh(\rho)$ to make sure $\rho$ is bounded below 1. 

In [12]:
#markup for leontieff

A= @. main.cost_m / main.rev
B= @. main.cost_l / (main.rev * beta_l_total_2)
markup_leontief= @. 1/(A+B)

mean_markup_leontief= mean(skipmissing(markup_leontief))
std_markup_leontief= std(skipmissing(markup_leontief))
median_markup_leontief= median(skipmissing(markup_leontief))

leontieff= vcat(mean_markup_leontief,std_markup_leontief, median_markup_leontief )


results_leontieff = DataFrame(
    Statistic = ["Mean", "Std. Dev.", "Median"],
    Markup_leontieff   = leontieff
)

show(stdout, MIME("text/latex"), results_leontieff)

\begin{tabular}{r|cc}
	& Statistic & Markup\_leontieff\\
	\hline
	& String & Float64\\
	\hline
	1 & Mean & 2.70228 \\
	2 & Std. Dev. & 23.2953 \\
	3 & Median & 1.16636 \\
\end{tabular}


14

**Exercise 3**


In [13]:
#we define another set of instruments and re-run all. 

Z_2 = hcat(
    ones(n),
    log_l_lag1,
    log_l_lag2,
    log_k_lag1,      # safer than log_k
    age_lag,
    acquired_lag2    
)

param2 = gmm(Z_2, n)

#impossible to identify! instrument acquired lag is not relevant; does not cary at all; this is clearly a weak IV problem. 

#compare GMM with OLS

table_gmm_ols2 = DataFrame(
    Parameter = ["β₀", "β_l", "β_k", "ψ", "β_a"],

    OLS_Estimate = [
        coef_ols[1],
        coef_ols[2],
        coef_ols[3],
        missing,
        missing
    ],

    OLS_SE = [
        se_ols[1],
        se_ols[2],
        se_ols[3],
        missing,
        missing
    ],

    GMM_Estimate = param[:, 1],
    GMM_SE       = param[:, 2],
    GMM2_Estimate = param2[:,1],
    GMM2_SE       = param2[:, 2]
)
show(stdout, MIME("text/latex"), table_gmm_ols2)


\begin{tabular}{r|ccccccc}
	& Parameter & OLS\_Estimate & OLS\_SE & GMM\_Estimate & GMM\_SE & GMM2\_Estimate & GMM2\_SE\\
	\hline
	& String & Float64? & Float64? & Float64 & Float64 & Float64 & Float64\\
	\hline
	1 & β₀ & -2.54257 & 0.319202 & -3.30106 & 0.282123 & 10037.2 & 3.86545e7 \\
	2 & β\_l & 0.401443 & 0.0460037 & 0.385612 & 0.0533172 & 0.144908 & 0.0976523 \\
	3 & β\_k & 0.822638 & 0.0507554 & 0.906453 & 0.060775 & -0.263957 & 0.12908 \\
	4 & ψ & \emph{missing} & \emph{missing} & -17.2182 & 1.0864e14 & 6.58 & 1928.35 \\
	5 & β\_a & \emph{missing} & \emph{missing} & 2.20214 & 0.951601 & 7.16234 & 12.2659 \\
\end{tabular}


14

In [14]:
#b, needs clean up wrt to 1...

using FixedEffectModels

total_cost = w .+  r

idx = findall(x -> !ismissing(x) && x > 0, total_cost)

markup = Vector{Union{Missing,Float64}}(missing, length(total_cost))
markup[idx] = rev[idx] ./ total_cost[idx]   # μ = revenue / total cost (your definition)

data.markup = markup

log_markup= log.(max.(markup, 1e-08))
data.log_markup = log.(max.(data.markup, 1e-8))  # avoid log(0)


df = dropmissing(data, [:log_markup, :acquired_lag1, :firmid, :yr])

reg_fe = reg(
    df,
    @formula(log_markup ~ acquired_lag1 + yr + fe(firmid)), 
    Vcov.cluster(:firmid)
)

 regtable(
    reg_fe;
    renderSettings = LatexTable(),
    file = joinpath(export_dir, "reg_fe_ex1.tex")
)
#simply can't again identify the effect of the merger as there is so little switchers...


┌ Warning: renderSettings is deprecated. Specify render type with render=LatexTable() and file with file=/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/reg_fe_ex1.tex
└ @ RegressionTables /Users/jorditorresvallverdu/.julia/packages/RegressionTables/wzLBu/src/regtable.jl:441


\begin{tabular}{lr}
\toprule
                     & \multicolumn{1}{c}{log\_markup} \\ 
\midrule
acquired\_lag1       &                          0.649* \\ 
                     &                         (0.277) \\ 
yr                   &                         0.114** \\ 
                     &                         (0.036) \\ 
\midrule
firmid Fixed Effects &                             Yes \\ 
\midrule
$N$                  &                             699 \\ 
$R^2$                &                           0.312 \\ 
Within-$R^2$         &                           0.032 \\ 
\bottomrule
\end{tabular}


In [15]:
#c

data.ratio= data.m ./ data.q

df = dropmissing(data, [:ratio, :acquired_lag1, :firmid, :yr])



reg_fe2 = reg(
    df,
    @formula(ratio ~ acquired_lag1 + yr + fe(firmid)),
    Vcov.cluster(:firmid)
)

 regtable(
    reg_fe2;
    renderSettings = LatexTable(),
    file = joinpath(export_dir, "reg_fe2_ex1.tex")
)

┌ Warning: renderSettings is deprecated. Specify render type with render=LatexTable() and file with file=/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/reg_fe2_ex1.tex
└ @ RegressionTables /Users/jorditorresvallverdu/.julia/packages/RegressionTables/wzLBu/src/regtable.jl:441


\begin{tabular}{lr}
\toprule
                     & \multicolumn{1}{c}{ratio} \\ 
\midrule
acquired\_lag1       &                 -0.237*** \\ 
                     &                   (0.044) \\ 
yr                   &                  -0.021** \\ 
                     &                   (0.007) \\ 
\midrule
firmid Fixed Effects &                       Yes \\ 
\midrule
$N$                  &                       883 \\ 
$R^2$                &                     0.508 \\ 
Within-$R^2$         &                     0.001 \\ 
\bottomrule
\end{tabular}


In [16]:
#c

data.input_price= log.(data.cost_m ./data.m)
df = dropmissing(data, [:input_price, :acquired_lag1, :firmid, :yr])



reg_fe3 = reg(
    df,
    @formula(input_price ~ acquired_lag1 + yr + fe(firmid)),
    Vcov.cluster(:firmid)
)


 regtable(
    reg_fe3;
    renderSettings = LatexTable(),
    file = joinpath(export_dir, "reg_fe3_ex1.tex")
)

┌ Warning: renderSettings is deprecated. Specify render type with render=LatexTable() and file with file=/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/reg_fe3_ex1.tex
└ @ RegressionTables /Users/jorditorresvallverdu/.julia/packages/RegressionTables/wzLBu/src/regtable.jl:441


\begin{tabular}{lr}
\toprule
                     & \multicolumn{1}{c}{input\_price} \\ 
\midrule
acquired\_lag1       &                         0.079*** \\ 
                     &                          (0.012) \\ 
yr                   &                         0.051*** \\ 
                     &                          (0.002) \\ 
\midrule
firmid Fixed Effects &                              Yes \\ 
\midrule
$N$                  &                              883 \\ 
$R^2$                &                            0.839 \\ 
Within-$R^2$         &                            0.666 \\ 
\bottomrule
\end{tabular}


results consistent with the paper but it is still puzzling how sensitive standard errors are to definition of cluster. Firms are basically more efficent than before and have more quality of products (pay more per input)

In [17]:
describe(data)



Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,Type
1,yr,1907.42,1898,1906.0,1920,0,Int64
2,prefid,9.11432,1,5.0,25,0,Int64
3,firmid,103.252,1,98.0,290,0,Int64
4,m,6.41244e5,1433,507173.0,6943344,2,"Union{Missing, Int64}"
5,k,1.94596e5,1450,136350.0,2945316,0,Int64
6,age,13.5316,2,12.0,38,0,Int64
7,acquired,0.00332963,0,0.0,1,0,Int64
8,cost_m,1.39262e6,2928.34,9.055e5,2.36546e7,2,"Union{Missing, Float64}"
9,q,1.46987e6,82.438,5.46572e5,2.20383e7,0,Float64


**Exercise 4**

Now we have to compute aggregate weighted by total input usage (labor+intermediate expenses)

In [18]:
using DataFrames, Statistics

function compute_weighted(data, y)

        df= copy(data)

        df.input_exp= df.cost_l .+ df.cost_m

        df= dropmissing(df, [y, :input_exp, :yr])


        g = groupby(df, :yr)

        out= combine(g, 

            y=>mean =>:mean,
            y=>median => :median, 
            [y, :input_exp]=>((o,x) -> begin
                w= x./sum(x)
                sum(w .* o)
            end)=> :agg
        )

         sort!(out, :yr)
        return out

end 

compute_weighted (generic function with 1 method)

In [19]:
##Compute productivity from GMM estimates 

#log_output - β0 - βl*log_l - βk*log_k
#    β0, βl, βk, ρ, βa = θ



data.productivity =
    log_output .- param[1] .- param[2] .* log_l .- param[3] .* log_k



data_productivity= compute_weighted(data, :productivity)

data_markup= compute_weighted(data, :markup)



Row,yr,mean,median,agg
,Int64,Float64,Float64,Float64
1,1898,8.93011,10.1529,10.2254
2,1899,9.88124,9.87951,10.7675
3,1900,10.486,9.88112,12.2143
4,1901,10.1942,9.16222,11.4275
5,1902,12.8047,12.4326,13.9156
6,1903,16.7748,13.8486,17.6093
7,1904,14.3086,12.2863,16.3243
8,1905,17.1854,14.3525,19.8548
9,1906,20.0027,17.3871,22.992


In [20]:
##plot !!!
using PyPlot

year        = data_productivity.yr
mean_val    = data_productivity.mean
median_val  = data_productivity.median
agg_val     = data_productivity.agg

figure(figsize=(6,4))

plot(year, mean_val,   label="Mean productivity", lw=2)
plot(year, median_val, label="Median productivity", lw=2, linestyle="--")
plot(year, agg_val,    label="Aggregate productivity", lw=3)

xlabel("Year")
ylabel("Productivity")
title("Productivity in the Japanese Cotton Industry")
legend()
grid(true)

tight_layout()

savefig(joinpath(export_dir, "productivity_time_series.png"), dpi=300)

close()   


##add standard deviations!!!


In [21]:
year   = data_markup.yr
mean_val  = data_markup.mean
median_val = data_markup.median
agg_val    = data_markup.agg

figure()
plot(year, mean_val,   label="Mean Markup",   lw=2)
plot(year, median_val, label="Median Markup", lw=2, linestyle="--")
plot(year, agg_val,    label="Aggregate Markup", lw=3)

xlabel("Year")
ylabel("Markup")
title("Markups in the Japanese Cotton Industry")
legend()
grid(true)

tight_layout()

savefig(joinpath(export_dir, "markup_time_series.png"), dpi=300)

close()   

##add standard deviations!!!